In [138]:
# Para llevar un registro de los cambios
%pip install -q comet_ml
import comet_ml
comet_ml.init(project_name='Tarea 03')

In [139]:
# Librerías
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [140]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)
experiment.set_name("Prueba 03.2")

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/387bbf019e624cb9901bdad1bfd0ca7c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     batch_accuracy [356] : (0.0, 0.9144278764724731)
COMET INFO:     batch_loss [356]     : (0.2861325442790985, 2.5217435359954834)
COMET INFO:   Others:
COMET INFO:     Name             : Prueba 03.2
COMET INFO:     trainable_params : 639300
COMET INFO:   Parameters:
COMET INFO:     RMSprop_centered                : False
COMET INFO:     RMSprop_clipnorm                : 1
COMET INFO:     RMSprop_clipvalue               : 1
COMET INFO:     RMSprop_ema_momentum            : 0.99
COMET INFO:     RMSprop_ema_overwrite_fre

In [141]:
# Carga el dataset de MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [142]:
# Convierte y normaliza los datos de entrada
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [143]:
# to_categorical convierte las etiquetas (train, test) en matrices one-hot
num_classes = 10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [144]:
  # Diccionario con los parámetros que se registrarán en comet
  parameters = {
      'epochs' : 40,
      'batch_size' : 15,
      'learning_rate' : 0.010,
      'loss' : 'categorical_crossentropy'
  }
  experiment.log_parameters(parameters)

In [145]:
# Creación del modelo
model = Sequential()
# Es como una "primera capa" de tres capas
model.add(Input(shape = (28,28))) # Matrices de entrada de 28x28 (imáegenes)
model.add(Flatten()) # Convierte las matrices de 28x28 en matrices de 784x1
model.add(Dense(784, activation = 'sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(30, activation = 'relu')) # Segunda capa
model.add(Dense(10, activation = 'softmax')) # Tercera capa

model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_12 (Flatten)        (None, 784)               0         
                                                                 
 dense_38 (Dense)            (None, 784)               615440    
                                                                 
 dropout_9 (Dropout)         (None, 784)               0         
                                                                 
 dense_39 (Dense)            (None, 30)                23550     
                                                                 
 dense_40 (Dense)            (None, 10)                310       
                                                                 
Total params: 639300 (2.44 MB)
Trainable params: 639300 (2.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [146]:
# Checkpoint que guarda el mejor modelo (cuando la función de costo alcanza un mínimo)
checkpoint = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_loss', verbose = 1, save_best_only = True, mode = 'min')

In [147]:
optimizer = RMSprop(learning_rate = 0.0005)
model.compile(loss = parameters['loss'], optimizer = optimizer, metrics = ['accuracy'])
model.fit(x_train, y_trainc, batch_size = parameters['batch_size'], epochs = parameters['epochs'], verbose = 1, validation_data = (x_test, y_testc), callbacks = [checkpoint])

Epoch 1/40
3996/4000 [============================>.] - ETA: 0s - loss: 0.4614 - accuracy: 0.8580
Epoch 1: val_loss improved from inf to 0.24109, saving model to best_model.hdf5
4000/4000 [==============================] - 21s 5ms/step - loss: 0.4610 - accuracy: 0.8581 - val_loss: 0.2411 - val_accuracy: 0.9285
Epoch 2/40
3993/4000 [============================>.] - ETA: 0s - loss: 0.2575 - accuracy: 0.9240
Epoch 2: val_loss improved from 0.24109 to 0.19224, saving model to best_model.hdf5
4000/4000 [==============================] - 21s 5ms/step - loss: 0.2573 - accuracy: 0.9241 - val_loss: 0.1922 - val_accuracy: 0.9451
Epoch 3/40
3991/4000 [============================>.] - ETA: 0s - loss: 0.2035 - accuracy: 0.9401
Epoch 3: val_loss improved from 0.19224 to 0.15294, saving model to best_model.hdf5
4000/4000 [==============================] - 21s 5ms/step - loss: 0.2034 - accuracy: 0.9402 - val_loss: 0.1529 - val_accuracy: 0.9542
Epoch 4/40
3994/4000 [============================>.] - 

In [148]:
experiment.log_model('MNIST1', 'best_model.hdf5')
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/723e4d25e7f346c9b1ffebd64db502c1
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [40]                  : (0.8581166863441467, 0.9846833348274231)
COMET INFO:     batch_accuracy [16000]         : (0.0, 1.0)
COMET INFO:     batch_loss [16000]             : (5.097576286061667e-05, 3.053478479385376)
COMET INFO:     epoch_duration [40]            : (20.222185921997152, 23.104500531000667)
COMET INFO:     loss [40]                      : (0.06658719480037689, 0.46103405952453613)
COMET INFO:     val_accuracy [40]              : (0.9284999966621399, 0.9825999736785889)
COMET INFO:     val_loss